In [ ]:
""" this notebook provides etl to extract and transform camps and clinics data"""

In [ ]:
import os
import pandas as pd

In [ ]:
from data_helper import get_csv_dataframes_from_folder
from data_helper import set_col_names_lower,set_col_names_underscore_separated
from data_helper import drop_unnamed_columns

In [ ]:
WRITE_FLAG=True

### Airtable data processing

In [ ]:
csv_dfs = get_csv_dataframes_from_folder('../data/raw/campsandclinincs/airtable')

In [ ]:
list_of_dfs = [df for file_name, df in csv_dfs.items()]


In [ ]:
airtable_df=pd.concat(list_of_dfs).drop_duplicates()
airtable_df.shape

In [ ]:
airtable_df_final=drop_unnamed_columns(set_col_names_underscore_separated(set_col_names_lower(airtable_df)))
airtable_df_final['websource']='airtable'
airtable_df_final

In [ ]:
airtable_df_final.to_csv('../data/campsandclinics/bronze/airtable.csv',index=False)

#### Mass Elite

In [ ]:
csv_dfs = get_csv_dataframes_from_folder('../data/raw/campsandclinincs/mass_elite/')
list(csv_dfs)

In [ ]:
list_dfs = [df.drop(df.columns[[0]],axis=1) for file_name, df in csv_dfs.items()]
mass_elite_df = pd.concat(list_dfs).drop_duplicates()
mass_elite_df

In [ ]:
mass_elite_df_final=drop_unnamed_columns(set_col_names_underscore_separated(set_col_names_lower(mass_elite_df)))
mass_elite_df_final=mass_elite_df_final.rename(columns={'camp_link':'link'})
mass_elite_df_final['websource']='masselite'
mass_elite_df_final

In [ ]:
mass_elite_df_final.to_csv('../data/campsandclinics/bronze/mass_elite.csv',index=False)

#### combine into one dataframe

In [ ]:
camps_clinics_df = pd.concat([airtable_df_final,mass_elite_df_final])
camps_clinics_df

#### Load into database

In [ ]:
#   open connection to database
#   load data into target database

In [ ]:
import mysql.connector

In [ ]:
# creds
host = '182.75.105.186'
dev_port = '33096'
dev_database = 'lacrosse-pre-dev'
password='L@crosse753'
user='lacrosse'

In [ ]:
# cnn = mysql.connector.connect(host =host, user = user, password = password, db = dev_database,port=dev_port)

In [ ]:
# cursor = cnn.cursor()

In [ ]:
# cursor = cnn.cursor()
# cursor.execute("SHOW DATABASES")
# for (databases) in cursor:
#     print(databases[0])

In [ ]:
from urllib.parse import quote
from sqlalchemy.engine import create_engine
engine = create_engine('mysql+mysqlconnector://lacrosse:%s@182.75.105.186:33096/lacrosse-pre-dev' % quote(password))

In [ ]:
# write data into table
camps_clinics_df.to_sql('camps_and_clinics',engine,if_exists='append',index=False)